In [2]:
import pandas as pd
from preprocessing.take_sentences import preprocess
from SimpleTransformers import NERModel
import os
from collections import Counter

/s/babbage/b/nobackup/nblancha/merry/conda/envs/mbert_ner/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
path="/s/red/a/nobackup/cwc-ro/shadim/languages/" #path to the NER folder for languages
run_num=1 #number of running code
lang_pairs=[['fr','br']
            ,['ar','fa']
            ,['ar','hi']
            ,['en','sco']
            ,['en','cy']
            ,['es','ca']
            ,['cs','sk']
            ,['id','ms']
            ,['fr','oc']
            ,['nl','af']
            ,['it','scn']
            ,['es','an']
            ,['es','ast']]
lang_codes={
    'fr':'french',
    'br':'breton',
    'ar':'arabic',
    'fa':'persian',
    'hi':'hindi',
    'en':'english',
    'sco':'scots',
    'cy':'welsh',
    'es':'spanish',
    'ca':'catalan',
    'cs':'czech',
    'sk':'slovak',
    'id':'indonesian',
    'ms':'malay',
    'oc':'occitan',
    'nl':'dutch',
    'af':'afrikan',
    'it':'italian',
    'scn':'sicilian',
    'an':'aragonese',
    'ast':'asturian'
}

In [5]:
language_accuracy=pd.DataFrame(columns=["l1","l2","l1-name","l2-name","model_type","total tokens of l1","total tokens of l2",
                                     "shared non-u in l1","shared non-u in l2","shared unique in both",
                                     "non-un token overlap in l1","non-un token overlap in l2",
                                    "eval_loss","precision","recall","f1_score"])

In [6]:
import logging
logging.disable(logging.WARNING)

In [7]:
size='17003'
run=1
analyzed_langs=[]
values_langs=[]
for l_index,lang_pair in lang_pairs:
    language_source,language_target=lang
    if(not language_source+"/"+language_target in analyzed_langs):
        analyzed_langs.append(language_source+"/"+language_target)
        outputdir_bert=path+'/ner_results_lan/'+language_source+'_'+language_target+'_bert'+str(run_num)
        outputdir_xlmr=path+'/ner_results_lan/'+language_source+'_'+language_target+'_xlmroberta'+str(run_num)

        path1=path+language_source+'/dataset_train'+str(run_num)+'.txt'
        path2=path+language_target+'/dataset_test'+str(run_num)+'.txt'

        f=open(path1,'r')
        sentences1=f.readlines()
        f=open(path2,'r')
        sentences2=f.readlines()
        sentences1_filtered=[]
        for sent in sentences1:
            if not 'None O' in sent and sent!='\n':
                sentences1_filtered.append(sent)
        sentences2_filtered=[]
        for sent in sentences2:
            if not 'None O' in sent and sent!='\n':
                sentences2_filtered.append(sent)       

        same_NERs=set(sentences1_filtered).intersection(set(sentences2_filtered))

        count1=Counter(sentences1_filtered)
        occur1=[count1[token] for token in same_NERs]
        all_occur1=sum(occur1)
        count2=Counter(sentences2_filtered)
        occur2=[count2[token] for token in same_NERs]
        all_occur2=sum(occur2)

        
        model_type='bert'
        model = NERModel(model_type, 
                    'bert-base-multilingual-cased',
                    labels=["O",
                    "B-MISC",
                    "I-MISC",
                    "B-PER",
                    "I-PER",
                    "B-ORG",
                    "I-ORG",
                    "B-LOC",
                    "I-LOC"],
                    use_cuda=True,
                    args={'save_model_every_epoch':False, 'save_steps': 10000,'output_dir':outputdir_bert, 'evaluate_during_training':True,'overwrite_output_dir':True, 'classification_report': True, 'save_eval_checkpoints':False,'cache_dir':path+'/ner_cache_dir'})

        trainingdataset=path+language_source+'/dataset_train'+str(run_num)+'.txt'
        testdataset=path+language_target+'/dataset_test'+str(run_num)+'.txt'
        results=model.train_model(trainingdataset,language_source,language_target,run,eval_df=testdataset)

        info=[language_source,language_target,lang_codes[language_source],lang_codes[language_target],model_type,len(sentences1_filtered),len(sentences2_filtered),
                        all_occur1,all_occur2,len(same_NERs),
                       all_occur1/len(sentences1_filtered),all_occur2/len(sentences2_filtered),
             results["eval_loss"],results["precision"],results["recall"],results["f1_score"]]
        print(info)
        language_accuracy.loc[len(language_accuracy.index)]=info

        model_type='xlmroberta'
        model = NERModel(model_type, 
                    'xlm-roberta-base',
                    labels=["O",
                    "B-MISC",
                    "I-MISC",
                    "B-PER",
                    "I-PER",
                    "B-ORG",
                    "I-ORG",
                    "B-LOC",
                    "I-LOC"],
                    use_cuda=True,
                    args={'save_model_every_epoch':False, 'save_steps': 10000,'output_dir':outputdir_xlmr, 'evaluate_during_training':True,'overwrite_output_dir':True, 'classification_report': True, 'save_eval_checkpoints':False,'cache_dir':path+'/ner_cache_dir'})

        trainingdataset=path+language_source+'/dataset_train'+str(run_num)+'.txt'
        testdataset=path+language_target+'/dataset_test'+str(run_num)+'.txt'
        results=model.train_model(trainingdataset,language_source,language_target,run,eval_df=testdataset)


        info=[language_source,language_target,lang_codes[language_source],lang_codes[language_target],model_type,len(sentences1_filtered),len(sentences2_filtered),
                        all_occur1,all_occur2,len(same_NERs),
                       all_occur1/len(sentences1_filtered),all_occur2/len(sentences2_filtered),
             results["eval_loss"],results["precision"],results["recall"],results["f1_score"]]
        print(info)
        language_accuracy.loc[len(language_accuracy.index)]=info

        language_source=lang2['l2']
        language_target=lang2['l2']
        
        path1=path+language_source+'/dataset_train'+str(run_num)+'.txt'
        path2=path+language_target+'/dataset_test'+str(run_num)+'.txt'

        f=open(path1,'r')
        sentences1=f.readlines()
        f=open(path2,'r')
        sentences2=f.readlines()
        sentences1_filtered=[]
        for sent in sentences1:
            if not 'None O' in sent and sent!='\n':
                sentences1_filtered.append(sent)
        sentences2_filtered=[]
        for sent in sentences2:
            if not 'None O' in sent and sent!='\n':
                sentences2_filtered.append(sent)       

        same_NERs=set(sentences1_filtered).intersection(set(sentences2_filtered))

        count1=Counter(sentences1_filtered)
        occur1=[count1[token] for token in same_NERs]
        all_occur1=sum(occur1)
        count2=Counter(sentences2_filtered)
        occur2=[count2[token] for token in same_NERs]
        all_occur2=sum(occur2)
        
        outputdir_bert=path+'/ner_results_lan/'+language_source+'_'+language_target+'_bert'+str(run_num)
        outputdir_xlmr=path+'/ner_results_lan/'+language_source+'_'+language_target+'_xlmroberta'+str(run_num)
        model_type='bert'
        model = NERModel(model_type, 
                    'bert-base-multilingual-cased',
                    labels=["O",
                    "B-MISC",
                    "I-MISC",
                    "B-PER",
                    "I-PER",
                    "B-ORG",
                    "I-ORG",
                    "B-LOC",
                    "I-LOC"],
                    use_cuda=True,
                    args={'save_model_every_epoch':False, 'save_steps': 10000,'output_dir':outputdir_bert, 'evaluate_during_training':True,'overwrite_output_dir':True, 'classification_report': True, 'save_eval_checkpoints':False,'cache_dir':path+'/ner_cache_dir'})

        trainingdataset=path+language_source+'/dataset_train'+str(run_num)+'.txt'
        testdataset=path+language_target+'/dataset_test'+str(run_num)+'.txt'
        results=model.train_model(trainingdataset,language_source,language_target,run,eval_df=testdataset)

        info=[language_target,language_target,lang_codes[language_target],lang_codes[language_target],model_type,len(sentences1_filtered),len(sentences2_filtered),
                        all_occur1,all_occur2,len(same_NERs),
                       all_occur1/len(sentences1_filtered),all_occur2/len(sentences2_filtered),
             results["eval_loss"],results["precision"],results["recall"],results["f1_score"]]
        print(info)
        language_accuracy.loc[len(language_accuracy.index)]=info

        model_type='xlmroberta'
        model = NERModel(model_type, 
                    'xlm-roberta-base',
                    labels=["O",
                    "B-MISC",
                    "I-MISC",
                    "B-PER",
                    "I-PER",
                    "B-ORG",
                    "I-ORG",
                    "B-LOC",
                    "I-LOC"],
                    use_cuda=True,
                    args={'save_model_every_epoch':False, 'save_steps': 10000,'output_dir':outputdir_xlmr, 'evaluate_during_training':True,'overwrite_output_dir':True, 'classification_report': True, 'save_eval_checkpoints':False,'cache_dir':path+'/ner_cache_dir'})

        trainingdataset=path+language_source+'/dataset_train'+str(run_num)+'.txt'
        testdataset=path+language_target+'/dataset_test'+str(run_num)+'.txt'
        results=model.train_model(trainingdataset,language_source,language_target,run,eval_df=testdataset)

        info=[language_target,language_target,lang_codes[language_target],lang_codes[language_target],model_type,len(sentences1_filtered),len(sentences2_filtered),
                        all_occur1,all_occur2,len(same_NERs),
                       all_occur1/len(sentences1_filtered),all_occur2/len(sentences2_filtered),
             results["eval_loss"],results["precision"],results["recall"],results["f1_score"]]
        print(info)
        language_accuracy.loc[len(language_accuracy.index)]=info

  0%|          | 0/13602 [00:00<?, ?it/s]

/s/babbage/b/nobackup/nblancha/merry/conda/envs/mbert_ner/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Current iteration:   0%|          | 0/1701 [00:00<?, ?it/s]

Running loss: 0.913873

...

/s/babbage/b/nobackup/nblancha/merry/conda/envs/mbert_ner/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.171509

  0%|          | 0/1120 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

['ast', 'ast', 'asturian', 'asturian', 'xlmroberta', 51310, 3309, 14039, 1577, 457, 0.2736113817969207, 0.4765790268963433, 0.16272627117162172, 0.8784722222222222, 0.9139072847682119, 0.8958394806727649]


In [13]:
language_accuracy.to_excel("all_langs_overlap_train_test.xlsx")language_accuracy.to_csv('all_langs_overlap_train_test', sep=',', index=False, encoding='utf-8')